In [1]:
import pandas as pd
import numpy as np
import joblib
from numbers import Number
from ta import momentum as mo
from ta import volume as vo
from ta import trend as tr
from sklearn import preprocessing as pp

In [2]:
ticker = 'zm'
hist_data_path = f'data/{ticker}_1.5y.csv'
hist_data = pd.read_csv(hist_data_path)
hist_data

,date,open,high,low,close,adj_close,volume
0,2020-03-02,109.349998,114.500000,102.500000,113.110001,113.110001,10157400
1,2020-03-03,114.550003,119.199997,112.110001,117.470001,117.470001,11033700
2,2020-03-04,118.510002,119.970001,113.110001,116.800003,116.800003,12102600
3,2020-03-05,110.250000,129.830002,110.000000,125.000000,125.000000,28149400
4,2020-03-06,119.650002,122.000000,107.000000,114.320000,114.320000,22975700
...,...,...,...,...,...,...,...
387,2021-09-13,300.160004,300.274994,289.820007,290.239990,290.239990,4916100
388,2021-09-14,290.338013,291.880005,280.920013,281.440002,281.440002,6036800
389,2021-09-15,282.329987,283.070007,277.019989,279.119995,279.119995,4647400
390,2021-09-16,277.679993,279.290009,273.500000,278.220001,278.220001,4676300


In [3]:
# ROC de Y em relação a X
def change(x, y):
    if len(x) == len(y):
        change_arr = []
        for i in range(0, len(x)):
            if isinstance(x[i], Number) and isinstance(y[i], Number):
                change_arr.append(((y[i] - x[i]) / x[i]) * 100)
            else:
                change_arr.append('NaN')
        
        return change_arr
    else:
        print(f"Error: x lenght {len(x)}, y lenght {len(y)}")

In [4]:
# print(change([1, 2],[2, 1])) # [100.0, -50.0] Yn > Xn se valor positivo
trend_window = 30
slice_start = 50
slice_plus = 0

roc = mo.roc(hist_data['close'], window=12)
rsi = mo.rsi(hist_data['close'])
stochrsi = mo.stochrsi(hist_data['close'])
stochrsi_d = mo.stochrsi_d(hist_data['close'])
stochrsi_k = mo.stochrsi_k(hist_data['close'])

tsi = mo.tsi(hist_data['close'])
tsi_signal = tr.ema_indicator(tsi, window=13)

macd = tr.macd(hist_data['close'])
macd_diff = tr.macd_diff(hist_data['close'])
macd_signal = tr.macd_signal(hist_data['close'])

stoch = mo.stoch(hist_data['high'], hist_data['low'], hist_data['close'])
ema_w8 = tr.ema_indicator(hist_data['close'], window=8)
ema_w20 = tr.ema_indicator(hist_data['close'], window=20)

sma_w13 = tr.sma_indicator(hist_data['close'], window=13)
sma_w25 = tr.sma_indicator(hist_data['close'], window=25)

roc_close_ema_w8 = change(ema_w8, hist_data['close'])
roc_ema_w8_ema_w20 = change(ema_w20, ema_w8)
roc_close_ema_w20 = change(ema_w20, hist_data['close'])
roc_close_open = change(hist_data['open'], hist_data['close'])

roc_close_sma_w13 = change(sma_w13, hist_data['close'])
roc_sma_w13_sma_w25 = change(sma_w25, sma_w13)
roc_close_sma_w25 = change(sma_w25, hist_data['close'])

trend = mo.roc(hist_data['close'], window=trend_window).map(lambda n : 'HIGH' if n > 0 else 'LOW')

df = pd.DataFrame({
    'close': np.array(list(hist_data['close'][slice_start:-(trend_window + slice_plus)])),
#     'roc': np.array(list(roc[slice_start:-(trend_window + slice_plus)])),
    'rsi': np.array(list(rsi[slice_start:-(trend_window + slice_plus)])),
    'stoch': np.array(list(stoch[slice_start:-(trend_window + slice_plus)])),
    'stochrsi': np.array(list(stochrsi[slice_start:-(trend_window + slice_plus)])),
    'stochrsi_d': np.array(list(stochrsi_d[slice_start:-(trend_window + slice_plus)])),
    'stochrsi_k': np.array(list(stochrsi_k[slice_start:-(trend_window + slice_plus)])),
    'macd': np.array(list(macd[slice_start:-(trend_window + slice_plus)])),
    'macd_diff': np.array(list(macd_diff[slice_start:-(trend_window + slice_plus)])),
    'macd_signal': np.array(list(macd_signal[slice_start:-(trend_window + slice_plus)])),
    'tsi': np.array(list(tsi[slice_start:-(trend_window + slice_plus)])),
    'tsi_signal': np.array(list(tsi_signal[slice_start:-(trend_window + slice_plus)])),
    'ema_w8': np.array(list(ema_w8[slice_start:-(trend_window + slice_plus)])),
    'ema_w20': np.array(list(ema_w20[slice_start:-(trend_window + slice_plus)])),
#     'roc_close_ema_w8': np.array(list(roc_close_ema_w8[slice_start:-(trend_window + slice_plus)])),
#     'roc_ema_w8_ema_w20': np.array(list(roc_ema_w8_ema_w20[slice_start:-(trend_window + slice_plus)])),
#     'roc_close_ema_w20': np.array(list(roc_close_ema_w20[slice_start:-(trend_window + slice_plus)])),
    
#     'roc_close_sma_w13': np.array(list(roc_close_sma_w13[slice_start:-(trend_window + slice_plus)])),
#     'roc_sma_w13_sma_w25': np.array(list(roc_sma_w13_sma_w25[slice_start:-(trend_window + slice_plus)])),
#     'roc_close_sma_w25': np.array(list(roc_close_sma_w25[slice_start:-(trend_window + slice_plus)])),
    'trend': np.array(list(trend[(slice_start + trend_window)::]))
})

# for n in df:
#     if n != 'trend':
#         df[n] = df[n].map(lambda x : round(x, 3))

df

,close,rsi,stoch,stochrsi,stochrsi_d,stochrsi_k,macd,macd_diff,macd_signal,tsi,tsi_signal,ema_w8,ema_w20,trend
0,161.039993,58.146040,58.099516,0.541349,0.536143,0.574870,5.934143,0.627869,5.306274,14.118075,14.204610,155.623541,149.958750,HIGH
1,167.139999,60.935702,70.591848,0.865508,0.619078,0.706229,6.586610,1.024269,5.562341,15.406991,14.376378,158.182754,151.595060,HIGH
2,167.910004,61.286475,90.754582,0.886599,0.681862,0.764485,7.084166,1.217460,5.866706,16.552289,14.687223,160.344365,153.148864,HIGH
3,174.830002,64.381856,98.573767,1.000000,0.796028,0.917369,7.945281,1.662859,6.282421,18.432642,15.222283,163.563395,155.213734,HIGH
4,164.690002,57.168684,74.865562,0.595694,0.836428,0.827431,7.720509,1.150471,6.570039,17.955950,15.612807,163.813752,156.116236,HIGH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,378.100006,55.574360,64.587867,0.737195,0.672245,0.775441,2.295182,0.504435,1.790747,5.091565,6.775463,372.057326,369.929599,LOW
308,378.959991,56.004476,66.208188,0.759490,0.768911,0.832228,2.770724,0.783982,1.986742,6.130163,6.683277,373.591252,370.789637,LOW
309,374.880005,53.364751,58.520977,0.622662,0.771373,0.706449,2.786256,0.639611,2.146645,6.189718,6.612768,373.877641,371.179195,LOW
310,400.579987,64.663261,95.910758,1.000000,0.777576,0.794051,4.816813,2.136135,2.680679,10.341624,7.145462,379.811496,373.979271,LOW


In [5]:
features = df.iloc[:,0:-1].values[-30:-25]
# features = pp.normalize(df.iloc[:,0:-1], axis=0, norm='l2')[-30:-25]
labels = df.iloc[:,-1].values[-30:-25]

In [6]:
loaded_rf = joblib.load(f"models/random_forest/{ticker}_rf.joblib")
corr_pred = 0
wrong_pred = 0

pred = loaded_rf.predict(features)

for i in list(range(0, len(pred))):
    
    if pred[i] == 'LOW' and labels[i] == 'LOW':
        corr_pred = corr_pred + 1
    elif pred[i] == 'HIGH' and labels[i] == 'HIGH':
        corr_pred = corr_pred + 1
    else:
        wrong_pred = wrong_pred + 1
    
print(f"Corret Prediction: {corr_pred}, {(corr_pred / len(pred)) * 100}%")
print(f"Wrong Prediction: {wrong_pred}, {(wrong_pred / len(pred)) * 100}%")

Corret Prediction: 2, 40.0%
Wrong Prediction: 3, 60.0%
